# Kubeflow Pipelines e2e mnist example

In this notebook you will create e2e mnist Kubeflow Pipeline to perform:
- Hyperparameter tuning using Katib
- Distributive training with the best hyperparameters using TFJob
- Serve the trained model using KServe

Reference documentation:

- https://www.kubeflow.org/docs/components/training/tftraining/
- https://www.kubeflow.org/docs/components/katib/
- https://www.kubeflow.org/docs/external-add-ons/kserve/

**Note**: This Pipeline runs in the multi-user mode. Follow [this guide](https://www.kubeflow.org/docs/components/pipelines/sdk/connect-api/#multi-user-mode) to give your Notebook access to Kubeflow Pipelines.

In [1]:
import kfp
import kfp.dsl as dsl
from kfp import components

from kubeflow.katib import ApiClient
from kubeflow.katib import V1beta1ExperimentSpec
from kubeflow.katib import V1beta1AlgorithmSpec
from kubeflow.katib import V1beta1ObjectiveSpec
from kubeflow.katib import V1beta1ParameterSpec
from kubeflow.katib import V1beta1FeasibleSpace
from kubeflow.katib import V1beta1TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec

## Define the Pipelines tasks

To run this Pipeline, you should define:
1. Katib hyperparameter tuning
2. TFJob training
3. KServe inference



### Step 1. Katib hyperparameter tuning task

Create the Kubeflow Pipelines task for the Katib hyperparameter tuning. This Experiment uses "random" algorithm and TFJob for the Trial's worker.

The Katib Experiment is similar to this example: https://github.com/kubeflow/katib/blob/master/examples/v1beta1/kubeflow-training-operator/tfjob-mnist-with-summaries.yaml.

### Step 2. TFJob training task

Create the Kubeflow Pipelines task for the TFJob training. In this example TFJob runs the Chief and Worker with 1 replica.

Learn more about TFJob replica specifications in the Kubeflow docs: https://www.kubeflow.org/docs/components/training/tftraining/#what-is-tfjob.

In [2]:
# You should define the TFJob name, namespace, number of training steps, output of Katib and model volume tasks in the arguments.
def create_tfjob_task(tfjob_name, tfjob_namespace, training_steps, model_volume_op):
    import json
    # Get parameters from the Katib Experiment.
    # Parameters are in the format "--tf-learning-rate=0.01 --tf-batch-size=100"
    #convert_katib_results_op = components.func_to_container_op(convert_katib_results)
    #best_hp_op = convert_katib_results_op(katib_op.output)
    #best_hps = str(best_hp_op.output)

    # Create the TFJob Chief and Worker specification with the best Hyperparameters.
    # TODO (andreyvelich): Use community image for the mnist example.
    tfjob_chief_spec = {
        "replicas": 1,
        "restartPolicy": "OnFailure",
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "tensorflow",
                        "image": "docker.io/sjseo85/category-model:1.6",
                        "command": [
                            "sh",
                            "-c"
                        ],
                        "args": [
                            "python /app/category_classification.py --tf-export-dir=/mnt/models/1 --tf-train-steps={} {}".format(1, 1)
                        ],
                        "volumeMounts": [
                            {
                                "mountPath": "/mnt/models",
                                "name": "model-volume"
                            }
                        ]
                    }
                ],
                "volumes": [
                    {
                        "name": "model-volume",
                        "persistentVolumeClaim": {
                            "claimName": str(model_volume_op.outputs["name"])
                        }
                    }
                ]
            }
        }
    }

    tfjob_worker_spec = {
        "replicas": 1,
        "restartPolicy": "OnFailure",
        "template": {
            "metadata": {
                "annotations": {
                    "sidecar.istio.io/inject": "false"
                }
            },
            "spec": {
                "containers": [
                    {
                        "name": "tensorflow",
                        "image": "docker.io/sjseo85/category-model:1.6",
                        "command": [
                            "sh",
                            "-c",
                        ],
                        "args": [
                          "python /app/category_classification.py --tf-export-dir=/mnt/models/1 --tf-train-steps={} {}".format(1, 1) 
                        ],
                    }
                ],
            }
        }
    }

    # Create the KFP task for the TFJob.
    tfjob_launcher_op = components.load_component_from_url(
        "https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/launcher/component.yaml")
    op = tfjob_launcher_op(
        name=tfjob_name,
        namespace=tfjob_namespace,
        chief_spec=json.dumps(tfjob_chief_spec),
        worker_spec=json.dumps(tfjob_worker_spec),
        tfjob_timeout_minutes=60,
        delete_finished_tfjob=False)
    return op

### Step 3. KServe inference

Create the Kubeflow Pipelines task for the KServe inference.

In [3]:
def create_serving_task(model_name, model_namespace, tfjob_op, model_volume_op):

    api_version = 'serving.kserve.io/v1beta1'
    serving_component_url = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kserve/component.yaml'

    # Uncomment the following two lines if you are using KFServing v0.6.x or v0.5.x
    # api_version = 'serving.kubeflow.org/v1beta1'
    # serving_component_url = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml'

    inference_service = '''
apiVersion: "{}"
kind: "InferenceService"
metadata:
  name: {}
  namespace: {}
  annotations:
    "sidecar.istio.io/inject": "false"
spec:
  predictor:
    tensorflow:
      storageUri: "pvc://{}/"
'''.format(api_version, model_name, model_namespace, str(model_volume_op.outputs["name"]))

    serving_launcher_op = components.load_component_from_url(serving_component_url)
    serving_launcher_op(action="apply", inferenceservice_yaml=inference_service).after(tfjob_op)

## Run the Kubeflow Pipeline

You should create the Kubeflow Pipeline from the above tasks.

In [5]:
name="category-e2e-v4"
namespace="kubeflow-user-example-com"
training_steps="100"

@dsl.pipeline(
    name="End to End Pipeline",
    description="An end to end mnist example including train and inference"
)
def category_pipeline(name=name, namespace=namespace, training_steps=training_steps):
    # Run the hyperparameter tuning with Katib.
    #katib_op = create_katib_experiment_task(name, namespace, training_steps)

    # Create volume to train and serve the model.
    model_volume_op = dsl.VolumeOp(
        name="model-volume",
        resource_name="model-volume",
        size="1Gi",
        modes=dsl.VOLUME_MODE_RWO
    )

    # Run the distributive training with TFJob.
    tfjob_op = create_tfjob_task(name, namespace, training_steps, model_volume_op)

    # Create the KServe inference.
    create_serving_task(name, namespace, tfjob_op, model_volume_op)
# Run the Kubeflow Pipeline in the user's namespace.

kfp_client=kfp.Client()
run_id = kfp_client.create_run_from_pipeline_func(category_pipeline, namespace=namespace, arguments={}).run_id
print("Run ID: ", run_id)

Run ID:  ba812e4d-079b-49a7-bb9b-d076ff74725f


## Predict from the trained model

Once Kubeflow Pipeline is finished, you are able to call the API endpoint with [mnist image](https://raw.githubusercontent.com/kubeflow/katib/master/examples/v1beta1/kubeflow-pipelines/images/9.bmp) to predict from the trained model.

**Note**: If you are using Kubeflow + Dex setup and runing this Notebook outside of your Kubernetes cluster, follow [this guide](https://github.com/kserve/kserve/tree/master/docs/samples/istio-dex#authentication) to get Session ID for the API requests.

In [12]:
import numpy as np
import requests
import json

# Pipeline Run should be succeeded.
kfp_run = kfp_client.get_run(run_id=run_id)
if kfp_run.run.status == "Succeeded":
    print("Run {} has been Succeeded\n".format(run_id))

    # Specify the image URL here.
    #image_url = "https://raw.githubusercontent.com/kubeflow/katib/master/examples/v1beta1/kubeflow-pipelines/images/9.bmp"
    #image = Image.open(requests.get(image_url, stream=True).raw)
    #data = np.array(image.convert('L').resize((28, 28))).astype(np.float).reshape(-1, 28, 28, 1)
    #data_formatted = np.array2string(data, separator=",", formatter={"float": lambda x: "%.1f" % x})
    #json_request = '{{ "instances" : {} }}'.format([[1, 2, 3, 4, 5, 2, 3, 4, 5, 2, 6]])
    json_request = {"instances":[[1,2,3,4]]}

    # Specify the prediction URL. If you are runing this notebook outside of Kubernetes cluster, you should set the Cluster IP.
    url = "http://{}-predictor-default.{}.svc.cluster.local/v1/models/{}:predict".format(name, namespace, name)
    print(url)
    #response = requests.post(url, data=json_request)
    
    r = requests.post(url="http://{}-predictor-default.{}.svc.cluster.local/v1/models/{}:predict".format(name, namespace, name), data=json.dumps(json_request))
    
    print("Prediction for the image")
    #display(image)
    print(r.json())

Run ba812e4d-079b-49a7-bb9b-d076ff74725f has been Succeeded

http://category-e2e-v4-predictor-default.kubeflow-user-example-com.svc.cluster.local/v1/models/category-e2e-v4:predict
Prediction for the image
{'predictions': [[0.00177408778, 0.00279735844, 0.00949931704, 0.00425323658, 0.00897183, 0.0152991116, 0.0113294674, 0.00479388749, 0.00690608, 0.0112169767, 0.00591368508, 0.00538556371, 0.00743126357, 0.00549162785, 0.0060344981, 0.00549076078, 0.0107562067, 0.0113362083, 0.0115286559, 0.00831412524, 0.0154880313, 0.00836658105, 0.00604901789, 0.00607525278, 0.011491931, 0.0101766856, 0.00667964667, 0.00838674605, 0.01140083, 0.00741297845, 0.0102966791, 0.00940914638, 0.00511750113, 0.00916701555, 0.00449514296, 0.0111307, 0.00605040928, 0.00642242283, 0.00648104772, 0.00858241785, 0.0126806693, 0.00433605397, 0.00526441121, 0.00459282193, 0.00922197849, 0.0123699559, 0.00613749167, 0.00869658496, 0.00440196833, 0.0110856593, 0.00883079413, 0.00710960617, 0.00805666484, 0.01207631